In [ ]:
# Resume Screening Automation - Complete Analysis
# This notebook performs end-to-end resume screening analysis

# ============================================================================
# CELL 1: Import Libraries
# ============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import warnings
warnings.filterwarnings('ignore')

# Import custom modules
import sys
sys.path.append('../src')

from data_preprocessing import ResumePreprocessor, load_and_preprocess_data
from feature_extraction import FeatureExtractor, ResumeMatcher

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Libraries imported successfully!")

# ============================================================================
# CELL 2: Load and Explore Data
# ============================================================================

# Load the dataset
df = pd.read_csv('../data/raw/Resume.csv')

print("Dataset Overview:")
print(f"Shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
print(df.head())

print(f"\n📊 Basic Statistics:")
print(f"Total Resumes: {len(df)}")
print(f"Unique Categories: {df['Category'].nunique()}")
print(f"Missing Values:\n{df.isnull().sum()}")

# ============================================================================
# CELL 3: Category Distribution Analysis
# ============================================================================

# Analyze category distribution
category_counts = df['Category'].value_counts()

print("\n📈 Category Distribution:")
print(category_counts)

# Visualize category distribution
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar plot
category_counts.plot(kind='bar', ax=axes[0], color='skyblue', edgecolor='black')
axes[0].set_title('Resume Count by Category', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Category', fontsize=12)
axes[0].set_ylabel('Count', fontsize=12)
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(axis='y', alpha=0.3)

# Pie chart (top 10 categories)
top_10_categories = category_counts.head(10)
axes[1].pie(top_10_categories, labels=top_10_categories.index, autopct='%1.1f%%', startangle=90)
axes[1].set_title('Top 10 Categories Distribution', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# ============================================================================
# CELL 4: Preprocess Resume Data
# ============================================================================

# Initialize preprocessor
preprocessor = ResumePreprocessor()

# Process the entire dataset
print("🔄 Preprocessing resumes...")
df_processed = preprocessor.process_dataframe(df, text_column='Resume_str')

print("\n✅ Preprocessing completed!")
print(f"New columns added: {[col for col in df_processed.columns if col not in df.columns]}")

# Display sample processed data
print("\n📝 Sample Processed Resume:")
print(f"Original (first 200 chars):\n{df_processed.iloc[0]['Resume_str'][:200]}")
print(f"\nCleaned (first 200 chars):\n{df_processed.iloc[0]['cleaned_resume'][:200]}")
print(f"\nExtracted Skills: {df_processed.iloc[0]['skills'][:10]}")

# ============================================================================
# CELL 5: Resume Length Analysis
# ============================================================================

# Analyze resume lengths
print("\n📏 Resume Length Statistics:")
print(df_processed[['resume_length', 'cleaned_length', 'num_skills']].describe())

# Visualize resume lengths
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Original resume length distribution
axes[0, 0].hist(df_processed['resume_length'], bins=50, color='coral', edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Distribution of Original Resume Length', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Number of Words')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].axvline(df_processed['resume_length'].mean(), color='red', linestyle='--', 
                    label=f"Mean: {df_processed['resume_length'].mean():.0f}")
axes[0, 0].legend()

# Cleaned resume length distribution
axes[0, 1].hist(df_processed['cleaned_length'], bins=50, color='lightgreen', edgecolor='black', alpha=0.7)
axes[0, 1].set_title('Distribution of Cleaned Resume Length', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Number of Words')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].axvline(df_processed['cleaned_length'].mean(), color='red', linestyle='--', 
                    label=f"Mean: {df_processed['cleaned_length'].mean():.0f}")
axes[0, 1].legend()

# Number of skills distribution
axes[1, 0].hist(df_processed['num_skills'], bins=30, color='skyblue', edgecolor='black', alpha=0.7)
axes[1, 0].set_title('Distribution of Identified Skills', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Number of Skills')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].axvline(df_processed['num_skills'].mean(), color='red', linestyle='--', 
                    label=f"Mean: {df_processed['num_skills'].mean():.1f}")
axes[1, 0].legend()

# Box plot: Skills by category (top 10 categories)
top_categories = df_processed['Category'].value_counts().head(10).index
df_top = df_processed[df_processed['Category'].isin(top_categories)]
df_top.boxplot(column='num_skills', by='Category', ax=axes[1, 1], patch_artist=True)
axes[1, 1].set_title('Skills Distribution by Category (Top 10)', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Category')
axes[1, 1].set_ylabel('Number of Skills')
plt.suptitle('')  # Remove default title

plt.tight_layout()
plt.show()

# ============================================================================
# CELL 6: Skills Analysis
# ============================================================================

# Extract all skills and count frequencies
from collections import Counter

all_skills = []
for skills_list in df_processed['skills']:
    all_skills.extend(skills_list)

skill_counts = Counter(all_skills)
top_skills = skill_counts.most_common(20)

print("\n🎯 Top 20 Skills Across All Resumes:")
for skill, count in top_skills:
    print(f"{skill}: {count}")

# Visualize top skills
skills_df = pd.DataFrame(top_skills, columns=['Skill', 'Count'])

plt.figure(figsize=(12, 6))
plt.barh(skills_df['Skill'], skills_df['Count'], color='teal', edgecolor='black')
plt.xlabel('Frequency', fontsize=12)
plt.ylabel('Skill', fontsize=12)
plt.title('Top 20 Most Common Skills', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

# ============================================================================
# CELL 7: Word Cloud Visualization
# ============================================================================

# Create word cloud from all cleaned resumes
all_text = ' '.join(df_processed['cleaned_resume'].astype(str))

wordcloud = WordCloud(width=1600, height=800, 
                      background_color='white', 
                      colormap='viridis',
                      max_words=100).generate(all_text)

plt.figure(figsize=(16, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of All Resumes', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Word clouds by category (top 4 categories)
top_4_categories = df_processed['Category'].value_counts().head(4).index

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, category in enumerate(top_4_categories):
    category_text = ' '.join(df_processed[df_processed['Category'] == category]['cleaned_resume'].astype(str))
    
    wc = WordCloud(width=800, height=400, 
                   background_color='white', 
                   colormap='plasma',
                   max_words=50).generate(category_text)
    
    axes[idx].imshow(wc, interpolation='bilinear')
    axes[idx].axis('off')
    axes[idx].set_title(f'Word Cloud: {category}', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

# ============================================================================
# CELL 8: TF-IDF Feature Extraction
# ============================================================================

# Initialize feature extractor
extractor = FeatureExtractor(max_features=1000, ngram_range=(1, 2))

# Fit and transform
print("🔧 Extracting TF-IDF features...")
tfidf_matrix, encoded_labels = extractor.fit_transform(
    df_processed['cleaned_resume'].tolist(),
    df_processed['Category'].tolist()
)

print(f"\n✅ TF-IDF extraction completed!")
print(f"Feature matrix shape: {tfidf_matrix.shape}")
print(f"Number of unique labels: {len(set(encoded_labels))}")

# Get feature names
feature_names = extractor.get_feature_names()
print(f"\nSample features: {feature_names[:20]}")

# ============================================================================
# CELL 9: Top TF-IDF Features by Category
# ============================================================================

# Function to get top features for each category
def get_category_top_features(df, extractor, category, top_n=15):
    """Get top TF-IDF features for a specific category"""
    category_resumes = df[df['Category'] == category]['cleaned_resume'].tolist()
    combined_text = ' '.join(category_resumes)
    return extractor.get_top_features(combined_text, top_n=top_n)

# Analyze top 3 categories
top_3_categories = df_processed['Category'].value_counts().head(3).index

print("\n🔍 Top TF-IDF Features by Category:\n")
for category in top_3_categories:
    print(f"\n{'='*60}")
    print(f"Category: {category}")
    print(f"{'='*60}")
    
    top_features = get_category_top_features(df_processed, extractor, category, top_n=15)
    
    for feature, score in top_features:
        print(f"  {feature:30s} : {score:.4f}")

# Visualize top features for each category
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for idx, category in enumerate(top_3_categories):
    top_features = get_category_top_features(df_processed, extractor, category, top_n=10)
    features, scores = zip(*top_features)
    
    axes[idx].barh(features, scores, color='purple', alpha=0.7, edgecolor='black')
    axes[idx].set_xlabel('TF-IDF Score', fontsize=10)
    axes[idx].set_title(f'{category}', fontsize=12, fontweight='bold')
    axes[idx].invert_yaxis()
    axes[idx].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

# ============================================================================
# CELL 10: Resume-Job Matching Example
# ============================================================================

# Initialize matcher
matcher = ResumeMatcher(extractor)

# Sample job descriptions for different categories
job_descriptions = {
    'Data Science': """
        Looking for a Data Scientist with strong Python programming skills.
        Must have experience with machine learning, deep learning, TensorFlow, and scikit-learn.
        Knowledge of SQL, pandas, numpy, and data visualization is required.
        Experience with AWS or cloud platforms is a plus.
    """,
    
    'Web Development': """
        Seeking a Full Stack Web Developer proficient in JavaScript, React, and Node.js.
        Experience with HTML, CSS, and responsive design required.
        Knowledge of databases (SQL, MongoDB) and RESTful APIs essential.
        Familiarity with Git, Agile methodologies, and CI/CD is preferred.
    """,
    
    'Java Developer': """
        Looking for Java Developer with expertise in Spring Boot and microservices.
        Must have experience with Docker, Kubernetes, and cloud deployment.
        Strong knowledge of SQL, REST APIs, and software design patterns required.
        Experience with Jenkins, Git, and Agile development is a plus.
    """
}

# Test matching for Data Science category
print("\n🎯 Resume Matching Example: Data Science Position\n")
print("="*80)
print("Job Description:")
print(job_descriptions['Data Science'])
print("="*80)

# Get top 5 matches
top_matches = matcher.rank_candidates(
    df_processed[df_processed['Category'] == 'Data Science'],
    job_descriptions['Data Science'],
    top_n=5
)

print(f"\n🏆 Top 5 Candidate Matches:\n")
for idx, row in top_matches.iterrows():
    print(f"\nCandidate {idx}:")
    print(f"  Match Score: {row['match_percentage']:.2f}%")
    print(f"  Skills: {row['skills'][:10]}")
    print(f"  Resume Preview: {row['Resume_str'][:150]}...")
    print("-"*80)

# ============================================================================
# CELL 11: Detailed Match Analysis
# ============================================================================

# Get detailed analysis for top candidate
if len(top_matches) > 0:
    top_candidate = top_matches.iloc[0]
    
    analysis = matcher.get_match_analysis(
        top_candidate['cleaned_resume'],
        job_descriptions['Data Science']
    )
    
    print("\n🔬 Detailed Match Analysis for Top Candidate:\n")
    print(f"Overall Match: {analysis['match_percentage']:.2f}%")
    print(f"\n✅ Matching Keywords ({len(analysis['matching_keywords'])}):")
    print(", ".join(analysis['matching_keywords'][:15]))
    
    print(f"\n❌ Missing Keywords ({len(analysis['missing_keywords'])}):")
    print(", ".join(analysis['missing_keywords'][:15]))
    
    print(f"\n📊 Candidate's Top Keywords:")
    for keyword, score in analysis['resume_top_keywords'][:10]:
        print(f"  {keyword:25s} : {score:.4f}")
    
    print(f"\n📋 Job's Top Keywords:")
    for keyword, score in analysis['job_top_keywords'][:10]:
        print(f"  {keyword:25s} : {score:.4f}")

# ============================================================================
# CELL 12: Cross-Category Matching Analysis
# ============================================================================

# Test how resumes from different categories match with a specific job
print("\n📊 Cross-Category Matching Analysis\n")
print("="*80)

categories_to_test = ['Data Science', 'Java Developer', 'Web Designing']
test_job = job_descriptions['Data Science']

results = []

for category in categories_to_test:
    category_resumes = df_processed[df_processed['Category'] == category].head(50)
    
    if len(category_resumes) > 0:
        similarities = extractor.batch_similarity(
            category_resumes['cleaned_resume'].tolist(),
            test_job
        )
        
        avg_similarity = similarities.mean()
        max_similarity = similarities.max()
        
        results.append({
            'Category': category,
            'Avg Match': avg_similarity * 100,
            'Max Match': max_similarity * 100
        })

results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

# Visualize cross-category matching
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(results_df))
width = 0.35

bars1 = ax.bar(x - width/2, results_df['Avg Match'], width, label='Average Match', color='steelblue', edgecolor='black')
bars2 = ax.bar(x + width/2, results_df['Max Match'], width, label='Max Match', color='coral', edgecolor='black')

ax.set_xlabel('Category', fontsize=12)
ax.set_ylabel('Match Percentage (%)', fontsize=12)
ax.set_title('Cross-Category Matching with Data Science Job', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(results_df['Category'])
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# ============================================================================
# CELL 13: Save Processed Data and Models
# ============================================================================

import os

# Create directories if they don't exist
os.makedirs('../data/processed', exist_ok=True)
os.makedirs('../models', exist_ok=True)

# Save processed dataframe
print("\n💾 Saving processed data and models...")

df_processed.to_csv('../data/processed/processed_resumes.csv', index=False)
print("✅ Processed data saved to: ../data/processed/processed_resumes.csv")

# Save vectorizer and label encoder
extractor.save_vectorizer('../models/tfidf_vectorizer.pkl')
extractor.save_label_encoder('../models/label_encoder.pkl')

print("\n🎉 Analysis complete! All models and data saved successfully!")

# ============================================================================
# CELL 14: Summary Statistics
# ============================================================================

print("\n" + "="*80)
print("📈 FINAL SUMMARY STATISTICS")
print("="*80)

summary = {
    'Total Resumes Processed': len(df_processed),
    'Total Categories': df_processed['Category'].nunique(),
    'Average Resume Length (words)': df_processed['resume_length'].mean(),
    'Average Skills per Resume': df_processed['num_skills'].mean(),
    'TF-IDF Features Extracted': tfidf_matrix.shape[1],
    'Most Common Category': df_processed['Category'].value_counts().index[0],
    'Top Skill Overall': max(skill_counts, key=skill_counts.get)
}

for key, value in summary.items():
    if isinstance(value, float):
        print(f"{key:.<50} {value:.2f}")
    else:
        print(f"{key:.<50} {value}")

print("="*80)

In [ ]:
#============================================================================

# ============================================================================
# CELL 15: Train Classification Models
# ============================================================================

from src.model_training import ResumeClassifier, train_and_evaluate_models
from sklearn.model_selection import train_test_split

print("="*80)
print("TRAINING CLASSIFICATION MODELS")
print("="*80)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    tfidf_matrix, 
    encoded_labels, 
    test_size=0.2, 
    random_state=42,
    stratify=encoded_labels
)

print(f"\nTraining set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

# Train and evaluate multiple models
results = train_and_evaluate_models(
    X_train, X_test, y_train, y_test,
    extractor.get_feature_names(),
    extractor.label_encoder
)

# ============================================================================
# CELL 16: Model Comparison Visualization
# ============================================================================

import plotly.graph_objects as go

# Extract metrics from results
models = list(results.keys())
metrics = ['accuracy', 'precision', 'recall', 'f1_score']

comparison_data = {
    metric: [results[model]['metrics'][metric] for model in models]
    for metric in metrics
}

# Create grouped bar chart
fig = go.Figure()

for metric in metrics:
    fig.add_trace(go.Bar(
        name=metric.capitalize(),
        x=models,
        y=comparison_data[metric],
        text=[f"{val:.3f}" for val in comparison_data[metric]],
        textposition='auto'
    ))

fig.update_layout(
    title='Model Performance Comparison',
    xaxis_title='Model',
    yaxis_title='Score',
    barmode='group',
    height=500
)

fig.show()

# Display numerical comparison
comparison_df = pd.DataFrame({
    model: {
        'Accuracy': results[model]['metrics']['accuracy'],
        'Precision': results[model]['metrics']['precision'],
        'Recall': results[model]['metrics']['recall'],
        'F1-Score': results[model]['metrics']['f1_score']
    }
    for model in results
}).T

print("\n📊 Numerical Comparison:")
print(comparison_df.round(4))

# Find best model
best_model = comparison_df['Accuracy'].idxmax()
print(f"\n🏆 Best Model: {best_model.upper()}")
print(f"   Accuracy: {comparison_df.loc[best_model, 'Accuracy']:.4f}")

# ============================================================================
# CELL 17: Confusion Matrix Visualization
# ============================================================================

import seaborn as sns

# Get best model
best_classifier = results[best_model]['classifier']
best_metrics = results[best_model]['metrics']

# Plot confusion matrix
plt.figure(figsize=(12, 10))

cm = best_metrics['confusion_matrix']
categories = extractor.label_encoder.classes_

sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=categories,
    yticklabels=categories,
    cbar_kws={'label': 'Count'}
)

plt.title(f'Confusion Matrix - {best_model.upper()}', fontsize=14, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# ============================================================================
# CELL 18: Feature Importance (Random Forest)
# ============================================================================

if best_model == 'random_forest':
    print("🌲 Random Forest Feature Importance")
    print("="*80)
    
    importance_df = best_classifier.get_feature_importance(
        extractor.get_feature_names(),
        top_n=20
    )
    
    print("\nTop 20 Most Important Features:")
    print(importance_df.to_string(index=False))
    
    # Visualize feature importance
    fig = px.bar(
        importance_df,
        x='importance',
        y='feature',
        orientation='h',
        title='Top 20 Feature Importance (Random Forest)',
        labels={'importance': 'Importance Score', 'feature': 'Feature'},
        color='importance',
        color_continuous_scale='Viridis'
    )
    
    fig.update_layout(yaxis={'categoryorder': 'total ascending'}, height=600)
    fig.show()

# ============================================================================
# CELL 19: Advanced Resume Matching
# ============================================================================

from src.resume_matcher import AdvancedResumeMatcher, JobDescriptionParser

print("="*80)
print("ADVANCED RESUME MATCHING")
print("="*80)

# Create advanced matcher
advanced_matcher = AdvancedResumeMatcher(extractor, best_classifier)

# Sample job description with requirements
job_description = """
Senior Data Scientist Position

We are seeking an experienced Data Scientist with 3-5 years of experience
to join our growing team. 

Required Skills:
- Python programming
- Machine Learning and Deep Learning
- TensorFlow, PyTorch, or scikit-learn
- SQL and database management
- Data visualization (Tableau, Power BI)
- AWS or Azure cloud platforms

Nice to Have:
- PhD or Master's degree in Computer Science, Statistics, or related field
- Experience with big data technologies (Spark, Hadoop)
- Natural Language Processing (NLP) experience
- Strong communication and teamwork skills

Responsibilities include building ML models, analyzing large datasets,
and collaborating with cross-functional teams.
"""

print("\n📋 Job Description:")
print(job_description[:200] + "...")

# Extract requirements from job description
required_skills = JobDescriptionParser.extract_required_skills(job_description)
experience_req = JobDescriptionParser.extract_experience_requirements(job_description)
education_req = JobDescriptionParser.extract_education_requirements(job_description)

print("\n🎯 Extracted Requirements:")
print(f"  Required Skills: {required_skills[:10]}")
print(f"  Experience: {experience_req['min_years']}-{experience_req['max_years']} years")
print(f"  Education: {education_req}")

# ============================================================================
# CELL 20: Hybrid Ranking
# ============================================================================

print("\n🔍 Performing Hybrid Ranking...")

# Filter to Data Science category
data_science_resumes = df_processed[df_processed['Category'] == 'Data Science']

# Perform hybrid ranking (TF-IDF + Skills)
top_candidates = advanced_matcher.hybrid_ranking(
    data_science_resumes,
    job_description,
    required_skills=required_skills,
    tfidf_weight=0.6,
    skill_weight=0.4,
    top_n=10
)

print(f"\n✅ Found {len(top_candidates)} top candidates")

# Display top 5
print("\n🏆 Top 5 Candidates:")
print("="*80)

for idx, (_, candidate) in enumerate(top_candidates.head(5).iterrows(), 1):
    print(f"\nCandidate #{idx}")
    print(f"  Overall Match: {candidate['match_percentage']:.2f}%")
    print(f"  TF-IDF Score: {candidate['tfidf_score']:.3f}")
    print(f"  Skill Score: {candidate['skill_score']:.3f}")
    print(f"  Total Skills: {candidate['num_skills']}")
    print(f"  Top Skills: {', '.join(candidate['skills'][:5])}")
    print("-"*80)

# ============================================================================
# CELL 21: Detailed Match Report
# ============================================================================

print("\n📄 Detailed Match Report for Top Candidate")
print("="*80)

# Get top candidate
top_candidate = top_candidates.iloc[0]

# Generate detailed report
detailed_report = advanced_matcher.get_detailed_match_report(
    top_candidate['cleaned_resume'],
    job_description,
    required_skills=required_skills
)

print(f"\n🎯 Match Score: {detailed_report['match_percentage']:.2f}%")
print(f"📊 Recommendation: {detailed_report['recommendation']}")

print(f"\n✅ Matching Keywords ({len(detailed_report['matching_keywords'])}):")
print(", ".join(detailed_report['matching_keywords'][:15]))

print(f"\n❌ Missing Keywords ({len(detailed_report['missing_keywords'])}):")
print(", ".join(detailed_report['missing_keywords'][:15]))

if detailed_report['skill_analysis']:
    print(f"\n🎓 Skill Analysis:")
    print(f"  Matched Skills ({len(detailed_report['skill_analysis']['matched_skills'])}):")
    print(f"    {', '.join(detailed_report['skill_analysis']['matched_skills'][:10])}")
    print(f"  Missing Skills ({len(detailed_report['skill_analysis']['missing_skills'])}):")
    print(f"    {', '.join(detailed_report['skill_analysis']['missing_skills'][:10])}")
    print(f"  Skill Match: {detailed_report['skill_analysis']['skill_match_percentage']:.1f}%")

print(f"\n📌 Top Resume Keywords:")
for keyword, score in detailed_report['resume_top_keywords'][:8]:
    print(f"  {keyword:25s} : {score:.4f}")

# ============================================================================
# CELL 22: Batch Candidate Analysis
# ============================================================================

print("\n📊 Batch Candidate Analysis")
print("="*80)

# Analyze all candidates
batch_results = advanced_matcher.batch_match_analysis(
    data_science_resumes,
    job_description,
    required_skills=required_skills,
    threshold=0.3
)

# Display summary
summary = batch_results['summary']

print(f"\n📈 Summary:")
print(f"  Total Candidates Analyzed: {summary['total_candidates']}")
print(f"  Strong Match (≥70%): {summary['strong_match_count']}")
print(f"  Good Match (50-70%): {summary['good_match_count']}")
print(f"  Moderate Match (30-50%): {summary['moderate_match_count']}")
print(f"  Weak Match (<30%): {summary['weak_match_count']}")

# Visualize distribution
categories_count = [
    summary['strong_match_count'],
    summary['good_match_count'],
    summary['moderate_match_count'],
    summary['weak_match_count']
]

category_labels = ['Strong Match', 'Good Match', 'Moderate Match', 'Weak Match']
colors = ['green', 'lightgreen', 'orange', 'red']

fig = go.Figure(data=[go.Pie(
    labels=category_labels,
    values=categories_count,
    hole=0.4,
    marker_colors=colors
)])

fig.update_layout(
    title='Candidate Distribution by Match Quality',
    height=500
)

fig.show()

# Display strong matches
if len(batch_results['strong_match']) > 0:
    print(f"\n🌟 Strong Match Candidates:")
    print("="*80)
    
    for idx, (_, candidate) in enumerate(batch_results['strong_match'].head(5).iterrows(), 1):
        print(f"\n{idx}. Match: {candidate['match_percentage']:.2f}%")
        print(f"   Skills: {', '.join(candidate['skills'][:5])}")

# ============================================================================
# CELL 23: Cross-Category Performance Analysis
# ============================================================================

print("\n🔍 Cross-Category Performance Analysis")
print("="*80)

# Test the same job on different categories
test_categories = ['Data Science', 'Information-Technology', 'Engineering']
category_performance = []

for category in test_categories:
    category_resumes = df_processed[df_processed['Category'] == category].head(100)
    
    if len(category_resumes) > 0:
        # Calculate similarities
        similarities = extractor.batch_similarity(
            category_resumes['cleaned_resume'].tolist(),
            job_description
        )
        
        category_performance.append({
            'Category': category,
            'Avg Match': similarities.mean() * 100,
            'Max Match': similarities.max() * 100,
            'Min Match': similarities.min() * 100,
            'Std Dev': similarities.std() * 100,
            'Median Match': np.median(similarities) * 100
        })

perf_df = pd.DataFrame(category_performance)
print("\n📊 Performance Metrics:")
print(perf_df.to_string(index=False))

# Visualize
fig = go.Figure()

fig.add_trace(go.Bar(
    name='Average Match',
    x=perf_df['Category'],
    y=perf_df['Avg Match'],
    marker_color='steelblue'
))

fig.add_trace(go.Bar(
    name='Max Match',
    x=perf_df['Category'],
    y=perf_df['Max Match'],
    marker_color='lightblue'
))

fig.update_layout(
    title='Cross-Category Match Performance',
    xaxis_title='Category',
    yaxis_title='Match Percentage (%)',
    barmode='group',
    height=500
)

fig.show()

# ============================================================================
# CELL 24: Save All Trained Models
# ============================================================================

import os

print("\n💾 Saving All Trained Models")
print("="*80)

# Create models directory
os.makedirs('models', exist_ok=True)

# Save all classification models
for model_name, model_data in results.items():
    model_path = f'models/{model_name}_model.pkl'
    model_data['classifier'].save_model(model_path)

# Save vectorizer and label encoder (if not already saved)
extractor.save_vectorizer('models/tfidf_vectorizer.pkl')
extractor.save_label_encoder('models/label_encoder.pkl')

print("\n✅ All models saved successfully!")
print("\nSaved models:")
print("  - models/tfidf_vectorizer.pkl")
print("  - models/label_encoder.pkl")
for model_name in results.keys():
    print(f"  - models/{model_name}_model.pkl")

# ============================================================================
# CELL 25: Final Summary and Recommendations
# ============================================================================

print("\n" + "="*80)
print("🎉 RESUME SCREENING AUTOMATION - COMPLETE SUMMARY")
print("="*80)

print("\n📊 Dataset Statistics:")
print(f"  Total Resumes: {len(df_processed):,}")
print(f"  Categories: {df_processed['Category'].nunique()}")
print(f"  Average Skills per Resume: {df_processed['num_skills'].mean():.1f}")
print(f"  Average Resume Length: {df_processed['resume_length'].mean():.0f} words")

print("\n🤖 Model Performance:")
print(f"  Best Model: {best_model.upper()}")
print(f"  Accuracy: {comparison_df.loc[best_model, 'Accuracy']:.4f}")
print(f"  F1-Score: {comparison_df.loc[best_model, 'F1-Score']:.4f}")

print("\n🎯 Resume Matching Results:")
print(f"  Top Match Score: {top_candidates.iloc[0]['match_percentage']:.2f}%")
print(f"  Average Top 10 Match: {top_candidates['match_percentage'].mean():.2f}%")
print(f"  Strong Matches Found: {summary['strong_match_count']}")

print("\n💡 Recommendations:")
print("  1. Use the best model for category prediction")
print("  2. Combine TF-IDF and skill matching for best results")
print("  3. Adjust matching thresholds based on position criticality")
print("  4. Regularly retrain models with new resume data")
print("  5. Use the Streamlit dashboard for interactive matching")

print("\n🚀 Next Steps:")
print("  1. Deploy models to production")
print("  2. Integrate with ATS systems")
print("  3. Set up automated email notifications")
print("  4. Create API endpoints for resume submission")
print("  5. Implement continuous model retraining")

print("\n" + "="*80)
print("✅ Analysis Complete! All models and data saved.")
print("="*80)